In [ ]:
# Prepare
import pandas as pd
import os, requests, matplotlib_venn, json
from matplotlib import pyplot as plt
os.chdir('/Users/denis/Documents/Projects/hiv-analysis/hiv-signature-analysis')
%load_ext rpy2.ipython
%R source('/Users/denis/Documents/Projects/scripts/Support.R')
%R library(ggplot2)

# Differential Expression Analysis

## Overview

The purpose of this notebook is to investigate the differential expression results of cell-line derived HIV-infected data, and the effect of the batch effect removal algorithm on these genes.

## Computation

In [ ]:
# Get infiles
infiles = ['f4-differential_expression.dir/podocyte_cell_line-vst-differential_expression.txt', 'f4-differential_expression.dir/podocyte_cell_line-vst_corrected-differential_expression.txt']

# Read data
cdDataframe = pd.read_table(infiles[0]).rename(columns={'CD':'CD_uncorrected'})
correctedCdDataframe = pd.read_table(infiles[1]).rename(columns={'CD':'CD_corrected'})

# Merge dataframes
mergedCdDataframe = cdDataframe.merge(correctedCdDataframe, on='gene_symbol', how='inner').set_index('gene_symbol')
mergedCdDataframe.head()

In [ ]:
# Get number of genes
nGenes = 500

# Get top genes
upregulatedGenes = {x:set(mergedCdDataframe[x].sort_values(ascending=False).index[:nGenes]) for x in mergedCdDataframe.columns}
downregulatedGenes = {x:set(mergedCdDataframe[x].sort_values(ascending=True).index[:nGenes]) for x in mergedCdDataframe.columns}
combinedGenes = {x:set(abs(mergedCdDataframe[x]).sort_values(ascending=False).index[:nGenes]) for x in mergedCdDataframe.columns}   

## Plots

### Scatter Plot

In [ ]:
%%R -i mergedCdDataframe
plot(mergedCdDataframe[,2:3],
     xlab = 'Original',
     ylab = 'Corrected',
     main = 'Comparison of Characteristic Direction Values\nOriginal vs Corrected Data')

### Venn Diagrams

In [ ]:
v = matplotlib_venn.venn2([upregulatedGenes['CD_uncorrected'], upregulatedGenes['CD_corrected']], set_labels=['Uncorrected','Corrected'])
plt.title('Top 500 Upregulated Genes')
plt.show()

In [ ]:
v = matplotlib_venn.venn2([downregulatedGenes['CD_uncorrected'], downregulatedGenes['CD_corrected']], set_labels=['Uncorrected','Corrected'])
plt.title('Top 500 Downregulated Genes')
plt.show()

In [ ]:
v = matplotlib_venn.venn2([combinedGenes['CD_uncorrected'], combinedGenes['CD_corrected']], set_labels=['Uncorrected','Corrected'])
plt.title('Top 500 Combined Genes')
plt.show()

## Analyses

### Enrichr

In [ ]:
# Define Enrichr API
def enrichr_get_url(genes, meta=''):
    """POST a gene list to Enrichr server and return the list ids"""
    genes_str = '\n'.join(genes)
    payload = {
        'list': (None, genes_str),
        'description': (None, meta)
    }
    # POST genes to the /addList endpoint
    response = requests.post("%s/addList" % 'http://amp.pharm.mssm.edu/Enrichr', files=payload)
    list_ids = json.loads(response.text)

    # Return URL
    result_url = 'http://amp.pharm.mssm.edu/Enrichr/enrich?dataset=' + list_ids['shortId']
    return result_url

In [ ]:
# Get geneset dict
genesetDict = {'Upregulated': upregulatedGenes, 'Downregulated': downregulatedGenes, 'Combined': combinedGenes}

# Get link dict
enrichrLinkDict = {dataset:{geneset:enrichr_get_url(genesetDict[dataset][geneset]) for geneset in genesetDict[dataset].keys()} for dataset in genesetDict.keys()}

In [ ]:
# Create dataframe
enrichrLinkDataframe = pd.DataFrame(enrichrLinkDict).T

# Display
pd.options.display.max_colwidth = 100
enrichrLinkDataframe

### L1000CDS<sup>2</sup>

In [ ]:
def runL1000CDS2(differentialExpressionDataframe, column):
    # Set data
    data = {"genes": differentialExpressionDataframe.index.tolist(), "vals":differentialExpressionDataframe[column].tolist()}
    data['genes'] = [x.upper() for x in data['genes']]
    
    # Set configuration
    config = {"aggravate":False, "searchMethod":"CD", "share":True, "combination":True, "db-version":"latest"}
    payload = {"data":data,"config":config}
    headers = {'content-type':'application/json'}
    
    # Perform request
    r = requests.post('http://amp.pharm.mssm.edu/L1000CDS2/query',data=json.dumps(payload),headers=headers)
    resCD= r.json()
    
    # Return result
    resultUrl = 'http://amp.pharm.mssm.edu/L1000CDS2/#/result/' + resCD['shareId']
    return resultUrl

In [ ]:
# Run analyses
l1000cds2Dict = {x: runL1000CDS2(mergedCdDataframe, x) for x in mergedCdDataframe.columns}

In [ ]:
# Convert to dataframe
l1000cds2Dataframe = pd.DataFrame.from_dict(l1000cds2Dict, orient='index').rename(columns={0:'L1000CDS2_link'})

# Display
l1000cds2Dataframe